# Package Imports

In [1]:
%matplotlib inline
from code.utils import *
from code.models import *

#To ensure multiple cell output
from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Set to False if not using Colab
google_colab = 'google.colab' in str(get_ipython())

if google_colab:
    
    #Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    #To replicate results locally —> change project_folder
    project_folder = Path("/content/drive/MyDrive/Bandit_Project/generated_data_4_project")
    input_file = project_folder/"generated4.csv"

else:
    project_folder = Path.cwd()
    input_file = Path("/Users/admin/Downloads/generated4.csv") #Change this location

# Importing the Data

In [3]:
#Initialize a control variable that determines whether we will import data
reexport_var = False #Do not change if experiment results are to replicated
hidden_include = False

train_path = project_folder/"train.csv"
val_path = project_folder/"val.csv"
test_path = project_folder/"test.csv"

In [4]:
if reexport_var:
    
  #This CSV file is generated by running DataGeneratory.ipynb - refer notebook for additional information  
  generate_CSV(input_file, train_path, val_path, test_path)

# Preparing Dataset for evaluation

In [5]:
batch_size=1024
n_epochs=500
feature_columns = ["user_id", "user_feature_1", "user_feature_2", "user_feature_hidden",
                   "campaign_id", "campaign_feature_1", "campaign_feature_2", "campaign_feature_hidden"]
target_column = "optimal_action"

train_dl = df_to_dataloader(train_path, feature_columns, target_column, batch_size=batch_size)
val_dl = df_to_dataloader(val_path, feature_columns, target_column, batch_size=batch_size)
test_dl = df_to_dataloader(test_path, feature_columns, target_column, shuffle = False, batch_size=batch_size)

print("[INFO] Train dataloader:")
pprint(train_dl)
print("[INFO] Val dataloader:")
pprint(val_dl)
print("[INFO] Test dataloader:")
pprint(test_dl)

[INFO] Train dataloader:
<BatchDataset shapes: ({user_id: (None,), user_feature_1: (None,), user_feature_2: (None,), user_feature_hidden: (None,), campaign_id: (None,), campaign_feature_1: (None,), campaign_feature_2: (None,), campaign_feature_hidden: (None,)}, (None, 10)), types: ({user_id: tf.int64, user_feature_1: tf.float64, user_feature_2: tf.float64, user_feature_hidden: tf.float64, campaign_id: tf.int64, campaign_feature_1: tf.float64, campaign_feature_2: tf.float64, campaign_feature_hidden: tf.float64}, tf.float32)>
[INFO] Val dataloader:
<BatchDataset shapes: ({user_id: (None,), user_feature_1: (None,), user_feature_2: (None,), user_feature_hidden: (None,), campaign_id: (None,), campaign_feature_1: (None,), campaign_feature_2: (None,), campaign_feature_hidden: (None,)}, (None, 10)), types: ({user_id: tf.int64, user_feature_1: tf.float64, user_feature_2: tf.float64, user_feature_hidden: tf.float64, campaign_id: tf.int64, campaign_feature_1: tf.float64, campaign_feature_2: tf.fl

# Creating TF Feature Columns

In [6]:
feature_column_dict, feature_column_input_dict = generate_feature_columns()

#Defining the inputs that will be fed to each model
inputs = {**feature_column_input_dict["numeric"], **feature_column_input_dict["embedding"]}

# Models

In [7]:
models_dir = project_folder/"models"
models_dir.mkdir(exist_ok=True)

#Create the folders to save the checkpoints
wmodel_dir = (models_dir/"Wide")
dmodel_dir = (models_dir/"Deep")
wdmodel_dir = (models_dir/"W&D")
bayesian_dir = (models_dir/"Bayesian")

wmodel_dir.mkdir(exist_ok=True)
dmodel_dir.mkdir(exist_ok=True)
wdmodel_dir.mkdir(exist_ok=True)
bayesian_dir.mkdir(exist_ok=True)

#Setting hyperparams
lr = 1e-3
gc.collect()

39

## Wide Only Model

In [8]:
wmodel, wmodel_path = build_wide_model(feature_column_dict, inputs, wmodel_dir)
wmodel.summary() #To display the architecture

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
campaign_feature_1 (InputLayer) [(None,)]            0                                            
__________________________________________________________________________________________________
campaign_feature_2 (InputLayer) [(None,)]            0                                            
__________________________________________________________________________________________________
campaign_id (InputLayer)        [(None,)]            0                                            
__________________________________________________________________________________________________
user_feature_1 (InputLayer)     [(None,)]            0                                            
______________________________________________________________________________________________

In [9]:
"""
#Training already done - Just load the model!
H = wmodel.fit(train_dl, batch_size=batch_size, epochs=n_epochs, 
               validation_data=val_dl, shuffle=False, 
               validation_batch_size=batch_size, callbacks=[w_2_es, w_2_mc])
"""               

'\n#Training already done - Just load the model!\nH = wmodel.fit(train_dl, batch_size=batch_size, epochs=n_epochs, \n               validation_data=val_dl, shuffle=False, \n               validation_batch_size=batch_size, callbacks=[w_2_es, w_2_mc])\n'

In [10]:
#The model has been trained before
wmodel = tf.keras.models.load_model(str(wmodel_path))

#Generate predictions on train, val & test set
eval_wmodel_train = wmodel.evaluate(train_dl)
eval_wmodel_val = wmodel.evaluate(val_dl)
eval_wmodel_test = wmodel.evaluate(test_dl)

#Print the results
print("\n[INFO] On Training Set:")
print(eval_wmodel_train)
print("\n[INFO] On Validation Set:")
print(eval_wmodel_val)
print("\n[INFO] On Test Set:")
print(eval_wmodel_test)

/Users/admin/miniforge3/envs/datascience/lib/python3.8/site-packages/tensorflow/python/keras/engine/functional.py:592: UserWarning: Input dict contained keys ['user_feature_hidden', 'campaign_feature_hidden'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


1954/1954 [==============================] - 198s 101ms/step - loss: 1.6896 - accuracy: 0.3335 - auc_1: 0.8280

[INFO] On Training Set:
[1.6183030605316162, 0.35337594151496887, 0.8419806957244873]

[INFO] On Validation Set:
[1.6896549463272095, 0.33308184146881104, 0.8280330896377563]

[INFO] On Test Set:
[1.6896229982376099, 0.3335239887237549, 0.8280342221260071]


## Deep Only Model

These are 4 different Deep-only model architectures that we use, namely:

1. With only embeddings
2. With only numeric features
3. With embeddings & numeric features
4. With both normal & hidden numeric features

For the sake of comparison, we employ the same `[512, 256, 128]` architecture.

In [11]:
#Model 1: Only Embeddings
dmodel_1_emb, dmodel_1_emb_path = build_deep_model(feature_column_dict["embedding"], inputs, dmodel_dir, 
                                                name="dmodel_1_emb.h5", ckpt_name="dmodel_1_emb_checkpoint.h5")

#Display summary to just show the results
dmodel_1_emb.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
campaign_feature_1 (InputLayer) [(None,)]            0                                            
__________________________________________________________________________________________________
campaign_feature_2 (InputLayer) [(None,)]            0                                            
__________________________________________________________________________________________________
campaign_id (InputLayer)        [(None,)]            0                                            
__________________________________________________________________________________________________
user_feature_1 (InputLayer)     [(None,)]            0                                            
____________________________________________________________________________________________

In [12]:
#Model 2: With only numeric features
dmodel_2_num, dmodel_2_num_path = build_deep_model(feature_column_dict["numeric"], inputs, dmodel_dir, 
                                                name="dmodel_2_num.h5", ckpt_name="dmodel_2_num_checkpoint.h5")
dmodel_2_num.summary()                                                

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
campaign_feature_1 (InputLayer) [(None,)]            0                                            
__________________________________________________________________________________________________
campaign_feature_2 (InputLayer) [(None,)]            0                                            
__________________________________________________________________________________________________
campaign_id (InputLayer)        [(None,)]            0                                            
__________________________________________________________________________________________________
user_feature_1 (InputLayer)     [(None,)]            0                                            
____________________________________________________________________________________________

In [13]:
#Model 3: With embeddings and numeric features
dmodel_3_num_emb, dmodel_3_num_emb_path = build_deep_model(feature_column_dict, inputs, dmodel_dir, 
                                                name="dmodel_3_num_emb.h5", ckpt_name="dmodel_3_num_emb_checkpoint.h5")
dmodel_3_num_emb.summary()                                                

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
campaign_feature_1 (InputLayer) [(None,)]            0                                            
__________________________________________________________________________________________________
campaign_feature_2 (InputLayer) [(None,)]            0                                            
__________________________________________________________________________________________________
campaign_id (InputLayer)        [(None,)]            0                                            
__________________________________________________________________________________________________
user_feature_1 (InputLayer)     [(None,)]            0                                            
____________________________________________________________________________________________

In [14]:
#Model 4: With normal & hidden numeric features
#Get the new feature column & input dicts
feature_column_dict_hidden, feature_column_input_dict_hidden = generate_feature_columns(hidden_include=True)
inputs_hidden = {**feature_column_input_dict_hidden["numeric"], **feature_column_input_dict_hidden["embedding"]}
dmodel_4_hid, dmodel_4_hid_path = build_deep_model(feature_column_dict_hidden, inputs_hidden, dmodel_dir, 
                                                    name="dmodel_4_hid.h5", ckpt_name="dmodel_4_hid_checkpoint.h5")
dmodel_4_hid.summary()                            

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
campaign_feature_1 (InputLayer) [(None,)]            0                                            
__________________________________________________________________________________________________
campaign_feature_2 (InputLayer) [(None,)]            0                                            
__________________________________________________________________________________________________
campaign_feature_hidden (InputL [(None,)]            0                                            
__________________________________________________________________________________________________
campaign_id (InputLayer)        [(None,)]            0                                            
____________________________________________________________________________________________

In [15]:
"""
#Training already done —> just load the models and evaluate!
H = dmodel_1_emb.fit(train_dl, batch_size=batch_size, epochs=n_epochs, 
               validation_data=val_dl, shuffle=False, 
               validation_batch_size=batch_size, callbacks=[d_es, d_mc])
"""               


'\n#Training already done —> just load the models and evaluate!\nH = dmodel_1_emb.fit(train_dl, batch_size=batch_size, epochs=n_epochs, \n               validation_data=val_dl, shuffle=False, \n               validation_batch_size=batch_size, callbacks=[d_es, d_mc])\n'

In [16]:
#For Deep Model 1: With Only Embeddings
dmodel_1_emb = tf.keras.models.load_model(str(dmodel_1_emb_path))

#Generate predictions on train, val & test set
eval_dmodel_1_emb_train = dmodel_1_emb.evaluate(train_dl, batch_size=batch_size)
eval_dmodel_1_emb_val = dmodel_1_emb.evaluate(val_dl, batch_size=batch_size)
eval_dmodel_1_emb_test = dmodel_1_emb.evaluate(test_dl, batch_size=batch_size)

#Print the results
print("\n[INFO] On Training Set:")
print(eval_dmodel_1_emb_train)
print("\n[INFO] On Validation Set:")
print(eval_dmodel_1_emb_val)
print("\n[INFO] On Test Set:")
print(eval_dmodel_1_emb_test)

1954/1954 [==============================] - 34s 18ms/step - loss: 1.8599 - accuracy: 0.2700 - auc_3: 0.7778

[INFO] On Training Set:
[1.8425158262252808, 0.27750545740127563, 0.7828749418258667]

[INFO] On Validation Set:
[1.8593494892120361, 0.270426869392395, 0.7780262231826782]

[INFO] On Test Set:
[1.8598766326904297, 0.27002349495887756, 0.7778425216674805]


In [17]:
#For Deep Model 2: With Only Numeric Features
dmodel_2_num = tf.keras.models.load_model(str(dmodel_2_num_path))

#Generate predictions on train, val & test set
eval_dmodel_2_num_train = dmodel_2_num.evaluate(train_dl, batch_size=batch_size)
eval_dmodel_2_num_val = dmodel_2_num.evaluate(val_dl, batch_size=batch_size)
eval_dmodel_2_num_test = dmodel_2_num.evaluate(test_dl, batch_size=batch_size)

#Print the results
print("\n[INFO] On Training Set:")
print(eval_dmodel_2_num_train)
print("\n[INFO] On Validation Set:")
print(eval_dmodel_2_num_val)
print("\n[INFO] On Test Set:")
print(eval_dmodel_2_num_test)

1954/1954 [==============================] - 35s 18ms/step - loss: 0.7128 - accuracy: 0.6781 - auc_2: 0.9690

[INFO] On Training Set:
[0.712095320224762, 0.6786260604858398, 0.9690924882888794]

[INFO] On Validation Set:
[0.7128351330757141, 0.6781636476516724, 0.9690284729003906]

[INFO] On Test Set:
[0.712820827960968, 0.6781215071678162, 0.9690293073654175]


In [18]:
#For Deep Model 3: With Embeddings + Numeric Features
dmodel_3_num_emb = tf.keras.models.load_model(str(dmodel_3_num_emb_path))

#Generate predictions on train, val & test set
eval_dmodel_3_num_emb_train = dmodel_3_num_emb.evaluate(train_dl, batch_size=batch_size)
eval_dmodel_3_num_emb_val = dmodel_3_num_emb.evaluate(val_dl, batch_size=batch_size)
eval_dmodel_3_num_emb_test = dmodel_3_num_emb.evaluate(test_dl, batch_size=batch_size)

#Print the results
print("\n[INFO] On Training Set:")
print(eval_dmodel_3_num_emb_train)
print("\n[INFO] On Validation Set:")
print(eval_dmodel_3_num_emb_val)
print("\n[INFO] On Test Set:")
print(eval_dmodel_3_num_emb_test)

1954/1954 [==============================] - 38s 19ms/step - loss: 0.4340 - accuracy: 0.8097 - auc_3: 0.9880

[INFO] On Training Set:
[0.4229640066623688, 0.8143868446350098, 0.9886316061019897]

[INFO] On Validation Set:
[0.4344477653503418, 0.8090035915374756, 0.9879717826843262]

[INFO] On Test Set:
[0.433987021446228, 0.8096619844436646, 0.9879959225654602]


In [19]:
#For Deep Model 4: With Both Hidden & Observable Numeric Features
dmodel_4_hid = tf.keras.models.load_model(str(dmodel_4_hid_path))

#Generate predictions on train, val & test set
eval_dmodel_4_hid_train = dmodel_4_hid.evaluate(train_dl, batch_size=batch_size)
eval_dmodel_4_hid_val = dmodel_4_hid.evaluate(val_dl, batch_size=batch_size)
eval_dmodel_4_hid_test = dmodel_4_hid.evaluate(test_dl, batch_size=batch_size)

#Print the results
print("\n[INFO] On Training Set:")
print(eval_dmodel_4_hid_train)
print("\n[INFO] On Validation Set:")
print(eval_dmodel_4_hid_val)
print("\n[INFO] On Test Set:")
print(eval_dmodel_4_hid_test)

1954/1954 [==============================] - 36s 19ms/step - loss: 0.0460 - accuracy: 0.9807 - auc_2: 0.9999

[INFO] On Training Set:
[0.045185185968875885, 0.9811403155326843, 0.9998612999916077]

[INFO] On Validation Set:
[0.04604890197515488, 0.9806995391845703, 0.9998518228530884]

[INFO] On Test Set:
[0.046024858951568604, 0.9807425141334534, 0.9998557567596436]


## Wide & Deep Model

Wide part receives the crossed columns.
Deep part receives the numeric features.

In [20]:
wdmodel, wdmodel_path = build_wide_and_deep_model(feature_column_dict, inputs, wdmodel_dir)
wdmodel.summary() #To display the architecture

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
campaign_feature_1 (InputLayer) [(None,)]            0                                            
__________________________________________________________________________________________________
campaign_feature_2 (InputLayer) [(None,)]            0                                            
__________________________________________________________________________________________________
campaign_id (InputLayer)        [(None,)]            0                                            
__________________________________________________________________________________________________
user_feature_1 (InputLayer)     [(None,)]            0                                            
____________________________________________________________________________________________

In [21]:
"""
#Model is already trained - load the model directly
wdmodel = tf.keras.models.load_model(str(wdmodel_checkpoint_path))
H = wdmodel.fit(train_dl, batch_size=batch_size, epochs=n_epochs, 
               validation_data=val_dl, shuffle=False, 
               validation_batch_size=batch_size, callbacks=[wd_es, wd_mc])
"""               

'\n#Model is already trained - load the model directly\nwdmodel = tf.keras.models.load_model(str(wdmodel_checkpoint_path))\nH = wdmodel.fit(train_dl, batch_size=batch_size, epochs=n_epochs, \n               validation_data=val_dl, shuffle=False, \n               validation_batch_size=batch_size, callbacks=[wd_es, wd_mc])\n'

In [22]:
#The model has been trained before
wdmodel = tf.keras.models.load_model(str(wdmodel_path))

#Generate predictions on train, val & test set
eval_wdmodel_train = wdmodel.evaluate(train_dl, batch_size=batch_size)
eval_wdmodel_val = wdmodel.evaluate(val_dl, batch_size=batch_size)
eval_wdmodel_test = wdmodel.evaluate(test_dl, batch_size=batch_size)

#Print the results
print("\n[INFO] On Training Set:")
print(eval_wdmodel_train)
print("\n[INFO] On Validation Set:")
print(eval_wdmodel_val)
print("\n[INFO] On Test Set:")
print(eval_wdmodel_test)

1954/1954 [==============================] - 478s 245ms/step - loss: 0.5485 - accuracy: 0.7735 - auc_1: 0.9805

[INFO] On Training Set:
[0.4669703245162964, 0.8039131760597229, 0.9863331317901611]

[INFO] On Validation Set:
[0.5484027862548828, 0.7732226252555847, 0.9805542230606079]

[INFO] On Test Set:
[0.5484840869903564, 0.7734814882278442, 0.9805416464805603]


# Summary of Results

This section reproduces Table 1 of the paper _Comparing the Performance of Deep and Wide vs. Deep Only Neural Networks_ from the results derived from the previous sections for the different model types and architectures. We use the `tabulate` package to print the table format.

In [23]:
l = [["Random", "none", "none", 10.0, 10.0, 10.0],
     ["Wide", "none", "user_id X campaign_id", f"{eval_wmodel_train[1]:.2f}", f"{eval_wmodel_val[1]:.2f}", f"{eval_wmodel_test[1]:.2f}"],
     ["Deep", "user and campaign id \nembeddings", "none", f"{eval_dmodel_1_emb_train[1]:.2f}", f"{eval_dmodel_1_emb_val[1]:.2f}", f"{eval_dmodel_1_emb_test[1]}"],
     ["Deep", "customer features 1&2,\ncampaign features 1&2", "none", f"{eval_dmodel_2_num_train[1]:.2f}", f"{eval_dmodel_2_num_val[1]:.2f}", f"{eval_dmodel_2_num_test[1]:.2f}"],
     ["Deep", "customer features 1&2,\ncampaign features 1&2, \nuser and campaign id \nembeddings", "none", f"{eval_dmodel_3_num_emb_train[1]:.2f}", f"{eval_dmodel_3_num_emb_val[1]:.2f}", f"{eval_dmodel_3_num_emb_test[1]:.2f}"],
     ["Deep", "customer features 1&2,\ncampaign features 1&2, \nhidden user and \ncampaign features", "none", f"{eval_dmodel_4_hid_train[1]:.2f}", f"{eval_dmodel_4_hid_val[1]:.2f}", f"{eval_dmodel_4_hid_test[1]:.2f}"],
     ["Wide & Deep", "customer features 1&2,\ncampaign features 1&2", "none", f"{eval_wdmodel_train[1]:.2f}", f"{eval_wdmodel_val[1]:.2f}", f"{eval_wdmodel_test[1]:.2f}"]]

table = tabulate(l, headers=['Model', 'Deep Input', 'Wide Input', 'Train\nAcc (%)', 'Val\nAcc (%)', 'Tst\nAcc (%)'], tablefmt='orgtbl')

print(table)

| Model       | Deep Input              | Wide Input            |     Train |       Val |       Tst |
|             |                         |                       |   Acc (%) |   Acc (%) |   Acc (%) |
|-------------+-------------------------+-----------------------+-----------+-----------+-----------|
| Random      | none                    | none                  |     10    |     10    | 10        |
| Wide        | none                    | user_id X campaign_id |      0.35 |      0.33 |  0.33     |
| Deep        | user and campaign id    | none                  |      0.28 |      0.27 |  0.270023 |
|             | embeddings              |                       |           |           |           |
| Deep        | customer features 1&2,  | none                  |      0.68 |      0.68 |  0.68     |
|             | campaign features 1&2   |                       |           |           |           |
| Deep        | customer features 1&2,  | none                  |      0.81 |     